In [ ]:
## Definitivo
import os
import glob
import numpy as np
import cftime
import h5py
import xesmf as xe
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from pyproj import Geod
from matplotlib.colors import LogNorm
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import json

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", module="xarray.coding.times")
warnings.filterwarnings("ignore", module="xarray.coding.cftime_offsets")
warnings.filterwarnings("ignore", category=DeprecationWarning, module="xarray")
warnings.filterwarnings("ignore", module="xarray")

import xarray as xr

# CONFIGURACIÓN DE RUTAS
bases_in = ["E:/TFG", "D:/TFG"]

# Definir el escenario
escenario = "SSP585"  

# Carpeta general donde guardar los resultados
BASE_RESULTADOS = "C:/Users/jaime/Desktop/Universidad/TFG/Resultados"

# Subcarpeta específica del escenario (para gráficos, mapas, etc.)
base_out = os.path.join(BASE_RESULTADOS, escenario, "CAT index")
# Crea la carpeta de salida si no existe
os.makedirs(base_out, exist_ok=True)
base_out2 = f"C:/Users/jaime/Desktop/Universidad/TFG/Resultados/{escenario}/zonal wind"
# Crea la carpeta de salida si no existe
os.makedirs(base_out2, exist_ok=True)

modelos = ["IPSL-CM6A-LR","HadGEM3-GC31-LL","CESM2-WACCM", "MPI-ESM1-2-LR", "NorESM2-LM","ACCESS-CM2",
"EC-Earth3-Veg-LR","MIROC6","TaiESM1"] ## poner todos los modelos; "GFDL-CM4"


# FUNCIÓN PARA ABRIR, INTERPOLAR Y CONCATENAR
def abrir_y_rejillar(archivos, varname, grid_out):
    datasets_proc = []
    regridder = None  # se inicializa una sola vez para ahorrar tiempo

    for f in archivos:
        #  Abrir evitando errores de calendario
        try:
            ds = xr.open_dataset(f, decode_times=False)
        except Exception:
            ds = xr.open_dataset(f, engine="h5netcdf", decode_times=False)

        # Decodificar CFTime (mantiene calendarios 360_day, noleap, etc.)
        if 'time' in ds.variables:
            ds = xr.decode_cf(ds, use_cftime=True)

        # Asegurar longitudes 0–360
        if (ds.lon < 0).any():
            ds = ds.assign_coords(lon=((ds.lon + 360) % 360))

        # Filtro estacional DJF
        if hasattr(ds['time'], 'dt'):
            ds = ds.sel(time=ds['time'].dt.month.isin([12, 1, 2]))
        else:
            print(f"⚠ {os.path.basename(f)}: eje temporal no tiene atributo .dt, se omite filtro DJF.")

        # Subdominio espacial y vertical
        ds = ds.sel(
            plev=slice(100000, 10000),
            lat=slice(25, 75),
            lon=slice(270, 10 + 360)
        )

        # Rejilla destino (solo se crea una vez)
        if regridder is None:
            regridder = xe.Regridder(ds, grid_out, method="bilinear", reuse_weights=False)

        ds_remap = regridder(ds)
        datasets_proc.append(ds_remap)

    # Concatenar archivos en el eje temporal
    ds_concat = xr.concat(datasets_proc, dim="time")

    # Filtrar periodo temporal 2030–2100
    ds_concat = ds_concat.where(
        (ds_concat["time"].dt.year >= 2030) & (ds_concat["time"].dt.year <= 2100),
        drop=True
    )

    # Mantener CFTimeIndex, sin convertir a pandas
    # (evita el error "Day out of range" y preserva calendarios CMIP)
    if not isinstance(ds_concat.indexes["time"], xr.coding.cftimeindex.CFTimeIndex):
        try:
            ds_concat = xr.decode_cf(ds_concat, use_cftime=True)
        except Exception as e:
            print(f"⚠ Aviso: no se pudo asegurar CFTimeIndex ({e})")
    # Extraer modelo desde la ruta y mostrar mensaje de progreso
    try:
        partes = os.path.normpath(f).split(os.sep)
        modelo = next((p for p in partes if p in modelos), "modelo_desconocido")
    except Exception:
        modelo = "modelo_desconocido"

    print(f"✔ Variable '{varname}' del modelo '{modelo}' ya procesada ({len(archivos)} archivos).")
    
    return ds_concat[varname]

# REJILLA DESTINO

grid_out = {
    "lon": np.arange(280, 355, 2.5),
    "lat": np.arange(35, 71, 2.0),
}


# FUNCIONES ADICIONALES

def asignar_invierno(ds):
    meses = ds['time'].dt.month
    años = ds['time'].dt.year
    winter_year = xr.where(meses == 12, años + 1, años)
    return ds.assign_coords(winter_year=("time", winter_year.data))

# FUNCIONES PARA DEF (TI1)
def _metric_derivs(u, v):
    R = 6_371_000.0
    deg2rad = np.pi / 180.0
    rad2deg = 180.0 / np.pi

    lat_rad = np.deg2rad(u['lat'])
    coslat  = np.cos(lat_rad)

    # Derivadas por grado (lo hace xarray)
    dudlon_deg = u.differentiate('lon')   # du/dλ (por grado)
    dudlat_deg = u.differentiate('lat')   # du/dφ (por grado)
    dvdlon_deg = v.differentiate('lon')
    dvdlat_deg = v.differentiate('lat')

    # Convertir "por grado" -> "por radian": multiplicar por 180/π
    ## Son derivadas por lo que tengo m/s/grados y quiero (m/s)/rad; por eso se multiplica por 180/pi
    dudlon_rad = dudlon_deg * rad2deg
    dudlat_rad = dudlat_deg * rad2deg
    dvdlon_rad = dvdlon_deg * rad2deg
    dvdlat_rad = dvdlat_deg * rad2deg

    # Pasar a derivadas espaciales (por metro)
    # ∂/∂x = (1/(R cosφ)) ∂/∂λ(rad) ; ∂/∂y = (1/R) ∂/∂φ(rad)
    dudx = dudlon_rad / (R * coslat)
    dudy = dudlat_rad / R
    dvdx = dvdlon_rad / (R * coslat)
    dvdy = dvdlat_rad / R

    return dudx, dudy, dvdx, dvdy


def compute_def(u, v):
    dudx, dudy, dvdx, dvdy = _metric_derivs(u, v)
    stretching = dudx - dvdy
    shearing   = dvdx + dudy
    return np.sqrt(stretching**2 + shearing**2)

## FUNCIONES PARA EL NÚMERO DE RICHARDSON GRADIENTE
def compute_theta(T, p):
    """Temperatura potencial θ [K]"""
    R_cp = 0.2856  # Aproximáción de K_p = R_aire/Cp_aire
    p0 = 100000.0  # Pa
    return T * (p0 / p)**R_cp

def compute_N2(theta, zg):
    g = 9.80665
    dtheta_dplev = theta.differentiate("plev")
    dz_dplev = zg.differentiate("plev")
    dtheta_dz = dtheta_dplev / dz_dplev
    N2 = (g / theta) * dtheta_dz
    # quita infinitos/nans raros
    return N2.where(np.isfinite(N2))

def compute_shear2(u, v, zg):
    dudz = u.differentiate("plev") / zg.differentiate("plev")
    dvdz = v.differentiate("plev") / zg.differentiate("plev")
    shear2 = dudz**2 + dvdz**2
    # evita valores ~0 que disparan Ri
    return shear2.where(shear2 > 1e-8)  # s^-2 (umbral conservador)

def compute_Ri(u, v, T, zg):
    theta = compute_theta(T, u["plev"])
    N2 = compute_N2(theta, zg)
    shear2 = compute_shear2(u, v, zg)

    # Filtrar solo estratificación estable
    N2 = N2.where(N2 > 0)

    Ri = N2 / shear2
    return Ri
    
# EJEMPLO RUTA TRANSATLÁNTICA
lon_DUB, lat_DUB = -6.2621, 53.4287   # Dublín
lon_JFK, lat_JFK = -73.7781, 40.6413  # Nueva York JFK
geod = Geod(ellps="WGS84")

In [ ]:
# 4. BUCLE PRINCIPAL: POR MODELO

import math
shear_slopes = []
VWS_mean_models = []
ti1_slopes = []
ti1_mean_models = []  # para guardar la media DJF de TI1 de cada modelo
ti1_signif_models = []  # para guardar las máscaras de significancia p<0.05
ti1_daily_all_models = []  # Guardará TI1 diario DJF
VWS_daily_all_models = []
Ri_slopes = []
ua_models = []
va_models = []
zg_models = []
ta_models = []

from matplotlib.colors import LinearSegmentedColormap
colors_ti1 = [
    "#fff7ec", "#fee8c8", "#fdd49e", "#fdbb84", 
    "#fc8d59", "#e34a33", "#b30000"
]
cmap_ti1 = LinearSegmentedColormap.from_list("TI1_soft", colors_ti1)

for modelo in modelos:
    print(f"Procesando {modelo}...")

    # Buscar archivos en una de las bases (E: o D:), no en ambas
    archivos_ua = archivos_va = archivos_zg = archivos_ta = []
    for base_in in bases_in:
        ruta_base = os.path.join(base_in, modelo, escenario)
        ruta_ua = os.path.join(ruta_base, "ua")
        ruta_va = os.path.join(ruta_base, "va")
        ruta_zg = os.path.join(ruta_base, "zg")
        ruta_ta = os.path.join(ruta_base, "ta")

        archivos_ua = sorted(glob.glob(os.path.join(ruta_ua, "*.nc")))
        archivos_va = sorted(glob.glob(os.path.join(ruta_va, "*.nc")))
        archivos_zg = sorted(glob.glob(os.path.join(ruta_zg, "*.nc")))
        archivos_ta = sorted(glob.glob(os.path.join(ruta_ta, "*.nc")))

        if archivos_ua and archivos_va and archivos_zg and archivos_ta:
            print(f"Archivos de {modelo} encontrados en {base_in}")
            break  # salir del bucle al encontrar el modelo completo
        else:
            print(f" {modelo} no completo en {base_in}, probando siguiente base...")

    # Si no se encuentra en ninguna base, saltar el modelo
    if not (archivos_ua and archivos_va and archivos_zg and archivos_ta):
        print(f" {modelo}: no se encontraron todos los archivos necesarios en ninguna base.")
        continue

    ua = abrir_y_rejillar(archivos_ua, "ua", grid_out)
    va = abrir_y_rejillar(archivos_va, "va", grid_out)
    zg = abrir_y_rejillar(archivos_zg, "zg", grid_out)
    ta = abrir_y_rejillar(archivos_ta, "ta", grid_out)

    # Filtros comunes: dominio y DJF
    for name, var in zip(["ua", "va", "zg", "ta"], [ua, va, zg, ta]):
        # Pasar longitudes a 0–360 si hiciera falta
        if (var.lon < 0).any():
            var = var.assign_coords(lon=((var.lon + 360) % 360))

        # Seleccionar invierno
        var = var.sel(time=var.time.dt.month.isin([12, 1, 2]))

        # Subdominio espacial
        var = var.sel(plev=slice(100000, 10000), lat=slice(34, 71), lon=slice(279, 360))

        globals()[name] = var  # re-asigna ua, va, zg, ta

    # Añadir a las listas con la nueva dimensión "modelo"
    ua_models.append(ua.expand_dims(modelo=[modelo]))
    va_models.append(va.expand_dims(modelo=[modelo]))
    zg_models.append(zg.expand_dims(modelo=[modelo]))
    ta_models.append(ta.expand_dims(modelo=[modelo]))

    # GUARDAR VARIABLES PROCESADAS INDIVIDUALMENTE (por modelo)

    # Ruta base donde guardar
    ruta_guardado = f"D:/TFG/Variables tratadas/{escenario}"
    os.makedirs(ruta_guardado, exist_ok=True)
    print(f"\n Guardando variables procesadas de {modelo}...")

    ruta_modelo = os.path.join(ruta_guardado, modelo)
    os.makedirs(ruta_modelo, exist_ok=True)

    # Elimina coord 'modelo' antes de guardar (para evitar conflictos)
    for var, name in zip([ua, va, zg, ta], ["ua", "va", "zg", "ta"]):
        if "modelo" in var.coords:
            var = var.drop_vars("modelo", errors="ignore")
        var.to_netcdf(os.path.join(ruta_modelo, f"{modelo}_{name}_DJF.nc"))

    print("\n Datos cargados y filtrados.")

    print(f"\n Calculando diagnósticos para {modelo}...")

    # CÁLCULOS DE VWS USANDO zg 
    u250 = ua.sel(plev=25000, method='nearest')
    u500 = ua.sel(plev=50000, method='nearest')
    v250 = va.sel(plev=25000, method='nearest')
    v500 = va.sel(plev=50000, method='nearest')
    dz = zg.sel(plev=25000, method='nearest') - zg.sel(plev=50000, method='nearest')
    if zg.plev.values[0] < zg.plev.values[-1]:
        dz = -dz  # corrige el signo si necesario

    # Evitar divisiones por cero o por capas demasiado delgadas
    dz = dz.where((dz > 100) & (dz < 10000))
    VWS = np.sqrt((u250 - u500)**2 + (v250 - v500)**2) / dz

    # GUARDAR VWS DIARIO DJF (para percentiles multimodelo)
    VWS_daily_all_models.append(VWS.assign_coords(modelo=modelo))
    VWS = VWS.where(np.isfinite(VWS))  # elimina inf o valores absurdos
    VWS = VWS.rename("VWS")

    VWS_with_year = asignar_invierno(VWS)
    VWS_djf_yearly = VWS_with_year.groupby("winter_year").mean("time")

    # Media climatológica DJF del TI1 (2030–2100)
    VWS_mean = VWS_djf_yearly.mean("winter_year")
    VWS_mean = VWS_mean.rename("VWS_mean")

    # Guardar para multimodelo
    VWS_mean = VWS_mean.assign_coords(modelo=modelo)
    VWS_mean_models.append(VWS_mean)

    # Graficar mapa climatológico del VWS (media de VWS)
    fig = plt.figure(figsize=(7, 4))
    ax = plt.axes(projection=ccrs.PlateCarree())
    vmax = float(VWS_mean.quantile(0.99))  # techo visual razonable
    norm = mcolors.Normalize(vmin=0, vmax=vmax)

    # Escalamos visualmente por 1e3 (para mostrar 1–10 en lugar de 0.001–0.01)
    (VWS_mean * 1e3).plot(
        ax=ax,
        transform=ccrs.PlateCarree(),
        cmap="YlOrRd",
        norm=mcolors.Normalize(vmin=0, vmax=vmax*1e3),
        cbar_kwargs={
            "label": "VWS medio DJF (×10⁻³ s⁻¹, 2030–2100)",
            "orientation": "horizontal",
            "pad": 0.1,
            "shrink": 0.7
        }
    )
    cs = ax.contour(
        VWS_mean * 1e3,
        levels=np.linspace(0, vmax*1e3, 8),
        colors='black',
        linewidths=0.3,
        transform=ccrs.PlateCarree()
    )
    ax.clabel(cs, fmt="%.1f", fontsize=6, inline=True)
    ax.coastlines(); ax.add_feature(cfeature.BORDERS, linewidth=0.5)
    ax.set_extent([280, 352.5, 35, 69], crs=ccrs.PlateCarree())

    # Añadir etiquetas de latitud y longitud 
    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                      linewidth=0.5, color='gray', alpha=0.5, linestyle='--')
    gl.top_labels = False
    gl.right_labels = False
    gl.xlabel_style = {"size": 8}
    gl.ylabel_style = {"size": 8}
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER

    plt.title(f"{modelo} – VWS medio DJF (2030–2100)\n{escenario}", fontsize=9)
    os.makedirs(base_out, exist_ok=True)
    plt.savefig(os.path.join(base_out, f"VWS_medio_{escenario}_{modelo}.png"),
                dpi=300, bbox_inches="tight")
    plt.close(fig)

    ######################################################################################
    VWS_trend = VWS_djf_yearly.polyfit(dim="winter_year", deg=1)
    VWS_slope = VWS_trend["polyfit_coefficients"].sel(degree=1) * 10
    VWS_slope = VWS_slope.assign_coords(modelo=modelo)
    shear_slopes.append(VWS_slope)

    print(f"\n {modelo}: diagnóstico VWS/TI1")
    print(f" - zg units: {zg.attrs.get('units', 'sin unidades')}")
    print(f" - plev: {ua.plev.values}")
    print(f" - zg500 media: {float(zg.sel(plev=50000, method='nearest').mean())}")
    print(f" - zg250 media: {float(zg.sel(plev=25000, method='nearest').mean())}")
    print(f" - Δz medio (m): {float((zg.sel(plev=25000, method='nearest') - zg.sel(plev=50000, method='nearest')).mean())}")
    print(f" - |u250-u500| media: {float(np.abs((u250 - u500).mean()))}")
    print(f" - |v250-v500| media: {float(np.abs((v250 - v500).mean()))}")
    print(f" - VWS min/max: {float(VWS.min())}/{float(VWS.max())}")

    # Detectar orden de niveles
    plev_vals = ua.plev.values
    ascendente = plev_vals[0] < plev_vals[-1]

    # Definir rango de presiones (Pa) más amplio y con sentido correcto
    if ascendente:
        plev_slice = slice(20000, 60000)
    else:
        plev_slice = slice(60000, 20000)

    # Seleccionar capa
    u_layer = ua.sel(plev=plev_slice)
    v_layer = va.sel(plev=plev_slice)
    zg_layer = zg.sel(plev=plev_slice)
    ta_layer = ta.sel(plev=plev_slice)

    print(f"{modelo}: niveles seleccionados -> {u_layer.plev.values}")

    if u_layer.plev.size < 2:
        print(f"{modelo}: solo {u_layer.plev.size} niveles dentro del rango {plev_slice}, se omite TI1")
        continue

    u_layer = ua.sel(plev=plev_slice)
    v_layer = va.sel(plev=plev_slice)
    zg_layer = zg.sel(plev=plev_slice)
    ta_layer = ta.sel(plev=plev_slice)

    # Calcular el término de deformación (shear tensor)
    DEF_layer = compute_def(u_layer, v_layer)

    p = u_layer["plev"]
    p_vals = p.values.astype(float)
    dp = np.gradient(p_vals)
    w = xr.DataArray(np.abs(dp), dims=["plev"], coords={"plev": p})

    DEF_layer = DEF_layer.where(np.isfinite(DEF_layer))
    DEF_bar = (DEF_layer * w).sum("plev") / w.sum("plev")

    # AÑADIR LOS PRINTS DE DIAGNÓSTICO
    print(f"{modelo}: VWS min/max = {float(VWS.min())}/{float(VWS.max())}")
    print(f"{modelo}: DEF_bar min/max = {float(DEF_bar.min())}/{float(DEF_bar.max())}")

    VWS = VWS.where(np.isfinite(VWS))
    DEF_bar = DEF_bar.where(np.isfinite(DEF_bar))
    DEF_bar = DEF_bar.clip(min=0)

    TI1 = (VWS * DEF_bar)
    TI1.attrs["units"] = "s⁻²"
    TI1 = TI1.clip(min=0, max=20e-7)
    TI1 = TI1.rename("TI1 (s^-2)")

    print(f"{modelo}: TI1 min/max = {float(TI1.min())}/{float(TI1.max())}")

    ti1_min = float(TI1.min())
    ti1_max = float(TI1.max())
    if not np.isfinite(ti1_min) or not np.isfinite(ti1_max) or ti1_max == 0:
        vabs = 1e-6
    else:
        vabs = max(abs(ti1_min), abs(ti1_max))
    vabs = max(vabs, 1e-7)
    norm = mcolors.TwoSlopeNorm(vmin=-vabs, vcenter=0, vmax=vabs)

    TI1_with_year = asignar_invierno(TI1)

    # GUARDAR VALORES DIARIOS DJF PARA PERCENTILES

    ti1_daily_all_models.append(TI1_with_year.assign_coords(modelo=modelo))
    TI1_djf_yearly = TI1_with_year.groupby("winter_year").mean("time")

    # Media climatológica DJF del TI1 (2030–2100)
    TI1_mean = TI1_djf_yearly.mean("winter_year")
    TI1_mean = TI1_mean.rename("TI1_mean")

    TI1_mean = TI1_mean.assign_coords(modelo=modelo)
    ti1_mean_models.append(TI1_mean)

    #################################################################################
    # Graficar mapa climatológico del TI1 (TI1 medio)
    fig = plt.figure(figsize=(7, 4))
    ax = plt.axes(projection=ccrs.PlateCarree())
    vmax = float(TI1_mean.quantile(0.98)) * 1e7
    vmin = float(TI1_mean.quantile(0.01)) * 1e7
    norm = mcolors.Normalize(vmin=vmin, vmax=vmax)
    cmap = plt.cm.get_cmap("YlOrRd")

    (TI1_mean * 1e7).plot(
        ax=ax,
        transform=ccrs.PlateCarree(),
        cmap=cmap,
        norm=norm,
        cbar_kwargs={
            "label": "TI1 medio DJF (x10⁻⁷ s⁻², 2030–2100)",
            "orientation": "horizontal",
            "pad": 0.1,
            "shrink": 0.7
        }
    )
    ax.coastlines(linewidth=0.7)
    ax.add_feature(cfeature.BORDERS, linewidth=0.4)
    ax.set_extent([280, 352.5, 35, 69], crs=ccrs.PlateCarree())
    plt.title(f"{modelo} – TI1 medio DJF (2030–2100)\n{escenario}", fontsize=9)
    plt.savefig(os.path.join(base_out, f"TI1_mean_{escenario}_{modelo}.png"),
                dpi=300, bbox_inches="tight")
    plt.close(fig)

    #######################################################################################
    TI1_trend = TI1_djf_yearly.polyfit(dim="winter_year", deg=1)
    TI1_slope = TI1_trend["polyfit_coefficients"].sel(degree=1) * 10
    TI1_slope = TI1_slope.assign_coords(modelo=modelo)
    ti1_slopes.append(TI1_slope)

    # Graficar tendencia de TI1 de cada modelo
    fig = plt.figure(figsize=(7, 4))
    ax = plt.axes(projection=ccrs.PlateCarree())
    lim = np.nanmax(np.abs(TI1_slope.values)) * 1e7
    norm = mcolors.TwoSlopeNorm(vmin=-lim, vcenter=0, vmax=lim)
    (TI1_slope * 1e7).plot(
        ax=ax,
        transform=ccrs.PlateCarree(),
        cmap="coolwarm",
        norm=norm,
        cbar_kwargs={
            "label": f"Tendencia TI1 {modelo} (x10⁻⁷ s⁻² por década)",
            "orientation": "horizontal",
            "pad": 0.1,
            "shrink": 0.7
        }
    )
    ax.coastlines()
    ax.add_feature(cfeature.BORDERS, linewidth=0.5)
    ax.set_extent([280, 352.5, 35, 69], crs=ccrs.PlateCarree())
    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                      linewidth=0.5, color='gray', alpha=0.5, linestyle='--')
    gl.top_labels = False
    gl.right_labels = False
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    gl.xlabel_style = {"size": 8}
    gl.ylabel_style = {"size": 8}
    plt.title(f"{modelo} – Tendencia TI1 (250–500 hPa) {escenario} ", fontsize=9)
    os.makedirs(base_out, exist_ok=True)
    plt.savefig(os.path.join(base_out, f"TI1_{modelo}.png"),
                dpi=300, bbox_inches="tight")
    plt.close(fig)



In [ ]:
########################################################################################
# MAPA DE DIFERENCIAS ENTRE CLASIFICACIONES TI₁ Y VWS
print("Generando mapa de diferencias entre categorías TI₁ y VWS...")
fig, ax = plt.subplots(subplot_kw={'projection': ccrs.PlateCarree()}, figsize=(7,4))
diff_map = diff_map_all_models.mean("modelo")
im = diff_map.plot(
    ax=ax, transform=ccrs.PlateCarree(), cmap="YlOrBr", vmin=0, vmax=10,
    cbar_kwargs={"label": "% de días con diferencia de categoría (TI₁ vs VWS)"}
)
ax.coastlines(); ax.add_feature(cfeature.BORDERS, linewidth=0.5)
ax.set_extent([280, 352.5, 35, 69], crs=ccrs.PlateCarree())
plt.title(f"Diferencias de clasificación TI₁ vs VWS – {escenario}", fontsize=10)
plt.tight_layout()
plt.savefig(os.path.join(base_out, f"diferencias_TI1_VWS_{escenario}.png"),
            dpi=300, bbox_inches="tight")
plt.close(fig)
print("✔ Mapa de diferencias TI₁–VWS guardado.")

# Frecuencias (promedio multimodelo; no existe 'time')
def freq_promedio_multimodelo(freq_ds_all):
    return {
        "Ligera - Ligera/Moderada": freq_ds_all["cat1"].mean("modelo"),
        "Moderada - Moderada/Severa": freq_ds_all["cat2"].mean("modelo"),
        "Severa": freq_ds_all["cat3"].mean("modelo"),
    }

freq_ti1 = freq_promedio_multimodelo(ti1_freq_all)
freq_vws = freq_promedio_multimodelo(vws_freq_all)
print("Frecuencias TI1 (%):", {k: float(v.mean()) for k, v in freq_ti1.items()})
print("Frecuencias VWS (%):", {k: float(v.mean()) for k, v in freq_vws.items()})

########################################################################################
# FIGURA CON TODOS LOS MAPAS DE VWS MEDIO DE LOS MODELOS

print("Creando figura combinada de VWS medio para todos los modelos...")
n_mod = len(VWS_mean_models)
if n_mod == 0:
    print("No hay VWS_mean_models disponibles. Se omite figura combinada de VWS.")
else:
    ncols = 2
    nrows = math.ceil(n_mod / ncols)
    fig, axes = plt.subplots(
        nrows, ncols, figsize=(ncols*3.5, nrows*3.0),
        subplot_kw={"projection": ccrs.PlateCarree()}
    )
    axes = axes.flatten()
    all_means = xr.concat(VWS_mean_models, dim="modelo")
    vmax_global_scaled = float((all_means * 1e3).quantile(0.99))
    norm_scaled = mcolors.Normalize(vmin=0, vmax=vmax_global_scaled)
    modelos_vws_validos = [da.modelo.values.item() for da in VWS_mean_models]
    last_i = -1
    for i, (modelo, VWS_map) in enumerate(zip(modelos_vws_validos, VWS_mean_models)):
        ax = axes[i]
        (VWS_map * 1e3).plot(
            ax=ax, transform=ccrs.PlateCarree(), cmap="YlOrRd",
            norm=norm_scaled, add_colorbar=False
        )
        try:
            cs = ax.contour(
                VWS_map * 1e3, levels=np.linspace(0, vmax_global_scaled, 8),
                colors='black', linewidths=0.3, transform=ccrs.PlateCarree()
            )
            ax.clabel(cs, fmt="%.1f", fontsize=5, inline=True)
        except Exception:
            pass
        ax.coastlines(); ax.add_feature(cfeature.BORDERS, linewidth=0.4)
        ax.set_extent([280, 352.5, 35, 69], crs=ccrs.PlateCarree())
        ax.set_title(modelo, fontsize=8)
        ax.set_xticks([]); ax.set_yticks([]); ax.set_xlabel(""); ax.set_ylabel("")
        last_i = i
    for j in range(last_i+1, len(axes)):
        fig.delaxes(axes[j])
    cbar_ax = fig.add_axes([0.25, 0.05, 0.5, 0.03])
    sm = plt.cm.ScalarMappable(cmap="YlGnBu", norm=norm_scaled)
    fig.colorbar(sm, cax=cbar_ax, orientation="horizontal",
                 label="VWS medio en DJF (×10⁻³ s⁻¹)")
    fig.suptitle(f"VWS medio DJF (2030–2100, {escenario}) – Todos los modelos", fontsize=11)
    plt.savefig(os.path.join(base_out, "VWS_mean_todos_modelos.png"),
                dpi=300, bbox_inches="tight")
    plt.close(fig)
print("✔ Figura combinada de VWS medio guardada.")

# FIGURA CON TODOS LOS MAPAS DE TI1 MEDIO DE LOS MODELOS
print("Creando figura combinada de TI1 medio para todos los modelos...")
n_mod = len(ti1_mean_models)
if n_mod == 0:
    print("No hay ti1_mean_models disponibles. Se omite figura combinada de TI1 medio.")
else:
    ncols = 2
    nrows = math.ceil(n_mod / ncols)
    fig, axes = plt.subplots(
        nrows, ncols, figsize=(ncols*3.5, nrows*3.0),
        subplot_kw={"projection": ccrs.PlateCarree()}
    )
    axes = axes.flatten()
    all_means = xr.concat(ti1_mean_models, dim="modelo")
    vmax_global_scaled = float((all_means * 1e7).quantile(0.99))
    norm_scaled = mcolors.Normalize(vmin=0, vmax=vmax_global_scaled)
    modelos_ti1_validos = [da.modelo.values.item() for da in ti1_mean_models]
    last_i = -1
    for i, (modelo, ti1_map) in enumerate(zip(modelos_ti1_validos, ti1_mean_models)):
        ax = axes[i]
        (ti1_map * 1e7).plot(
            ax=ax, transform=ccrs.PlateCarree(), cmap=cmap_ti1,
            norm=norm_scaled, add_colorbar=False
        )
        try:
            cs = ax.contour(
                ti1_map * 1e7, levels=np.linspace(0, vmax_global_scaled, 8),
                colors='black', linewidths=0.3, transform=ccrs.PlateCarree()
            )
            ax.clabel(cs, fmt="%.1f", fontsize=5, inline=True)
        except Exception:
            pass
        ax.coastlines(); ax.add_feature(cfeature.BORDERS, linewidth=0.4)
        ax.set_extent([280, 352.5, 35, 69], crs=ccrs.PlateCarree())
        ax.set_title(modelo, fontsize=8)
        ax.set_xticks([]); ax.set_yticks([]); ax.set_xlabel(""); ax.set_ylabel("")
        last_i = i
    for j in range(last_i+1, len(axes)):
        fig.delaxes(axes[j])
    cbar_ax = fig.add_axes([0.25, 0.05, 0.5, 0.03])
    sm = plt.cm.ScalarMappable(cmap=cmap_ti1, norm=norm_scaled)
    fig.colorbar(sm, cax=cbar_ax, orientation="horizontal",
                 label="TI1 medio DJF (×10⁻⁷ s⁻²)")
    fig.suptitle(f"TI1 medio DJF (2030–2100, {escenario}) – Todos los modelos", fontsize=11)
    plt.savefig(os.path.join(base_out, "TI1_mean_todos_modelos.png"),
                dpi=300, bbox_inches="tight")
    plt.close(fig)
print("✔ Figura combinada de TI1 medio guardada.")

# FIGURA CON TODAS LAS TENDENCIAS DE TI1 DE LOS MODELOS
print("Creando figura combinada de tendencias TI1 para todos los modelos...")
n_mod = len(ti1_slopes)
ncols = 2
nrows = math.ceil(n_mod / ncols)
fig, axes = plt.subplots(
    nrows, ncols, figsize=(ncols*3.5, nrows*3.0),
    subplot_kw={"projection": ccrs.PlateCarree()}
)
axes = axes.flatten()
all_slopes = xr.concat(ti1_slopes, dim="modelo")
lim_global_scaled = float((abs(all_slopes) * 1e7).quantile(0.99))
norm_scaled = mcolors.TwoSlopeNorm(vmin=-lim_global_scaled, vcenter=0, vmax=lim_global_scaled)
modelos_ti1trend_validos = [da.modelo.values.item() for da in ti1_slopes]
last_i = -1
for i, (modelo, ti1_slope) in enumerate(zip(modelos_ti1trend_validos, ti1_slopes)):
    ax = axes[i]
    (ti1_slope * 1e7).plot(
        ax=ax, transform=ccrs.PlateCarree(), cmap="RdBu_r",
        norm=norm_scaled, add_colorbar=False
    )
    try:
        cs = ax.contour(
            ti1_slope * 1e7,
            levels=np.linspace(-lim_global_scaled, lim_global_scaled, 9),
            colors='black', linewidths=0.3, transform=ccrs.PlateCarree()
        )
        ax.clabel(cs, fmt="%.2f", fontsize=5, inline=True)
    except Exception:
        pass
    ax.coastlines(); ax.add_feature(cfeature.BORDERS, linewidth=0.4)
    ax.set_extent([280, 352.5, 35, 69], crs=ccrs.PlateCarree())
    ax.set_title(modelo, fontsize=8)
    ax.set_xticks([]); ax.set_yticks([])
    last_i = i
for ax in axes.flat:
    ax.set_xticks([]); ax.set_yticks([]); ax.set_xlabel(""); ax.set_ylabel("")
for j in range(last_i+1, len(axes)):
    fig.delaxes(axes[j])
cbar_ax = fig.add_axes([0.25, 0.05, 0.5, 0.03])
sm = plt.cm.ScalarMappable(cmap="RdBu_r", norm=norm_scaled)
fig.colorbar(sm, cax=cbar_ax, orientation="horizontal",
             label="Tendencia TI1 (×10⁻⁷ s⁻² por década)")
fig.suptitle(f"Tendencia TI1 (250–500 hPa, 2030–2100, {escenario}) – Todos los modelos", fontsize=11)
plt.savefig(os.path.join(base_out, "TI1_tendencia_todos_modelos.png"),
            dpi=300, bbox_inches="tight")
plt.close(fig)
print("✔ Figura combinada de tendencias TI1 guardada.")
print("✔ Cálculo completado para TI1 y cizalladura. Pasando al análisis de Richardson...")


In [ ]:
# NÚMERO DE RICHARDSON — FRECUENCIA DE INESTABILIDAD (250–500 hPa)

Ri_daily_all_models = []
Ri_freq_5_models = []

for modelo in modelos:
    print(f"Procesando frecuencia de inestabilidad para {modelo}...")

    try:
        ua = [m for m in ua_models if modelo in m.modelo.values][0].squeeze("modelo")
        va = [m for m in va_models if modelo in m.modelo.values][0].squeeze("modelo")
        zg = [m for m in zg_models if modelo in m.modelo.values][0].squeeze("modelo")
        ta = [m for m in ta_models if modelo in m.modelo.values][0].squeeze("modelo")
    except IndexError:
        print(f" {modelo}: no se encontró en las listas preprocesadas.")
        continue

    # Filtrar DJF (por si acaso)
    ua = ua.sel(time=ua.time.dt.month.isin([12, 1, 2]))
    va = va.sel(time=va.time.dt.month.isin([12, 1, 2]))
    zg = zg.sel(time=zg.time.dt.month.isin([12, 1, 2]))
    ta = ta.sel(time=ta.time.dt.month.isin([12, 1, 2]))

    # Seleccionar niveles 250 y 500 hPa
    plev_vals = ua["plev"].values
    plev_250 = plev_vals[np.argmin(np.abs(plev_vals - 25000))]
    plev_500 = plev_vals[np.argmin(np.abs(plev_vals - 50000))]

    u250, u500 = ua.sel(plev=plev_250), ua.sel(plev=plev_500)
    v250, v500 = va.sel(plev=plev_250), va.sel(plev=plev_500)
    z250, z500 = zg.sel(plev=plev_250), zg.sel(plev=plev_500)
    T250, T500 = ta.sel(plev=plev_250), ta.sel(plev=plev_500)

    # Calcular Ri diario (versión corregida)
    g, p0, kappa = 9.80665, 100000.0, 0.2856
    
    # Alturas geopotenciales
    z250_val = zg.sel(plev=plev_250)
    z500_val = zg.sel(plev=plev_500)
    
    # Δz POSITIVO (m)
    dz = np.abs(z250_val - z500_val)
    dz = dz.where((dz > 100) & (dz < 10000))  # evitar outliers absurdos
    
    # Cizalla vertical
    dU = u250 - u500
    dV = v250 - v500
    shear2 = (dU**2 + dV**2) / dz**2
    shear2 = shear2.where(np.isfinite(dU + dV) & (shear2 > 1e-8))

    # Temperatura potencial
    theta250 = T250 * (p0 / plev_250) ** kappa
    theta500 = T500 * (p0 / plev_500) ** kappa
    theta_bar = 0.5 * (theta250 + theta500)
    
    # Gradiente de estabilidad (mantiene signo)
    dtheta_dz = (theta250 - theta500) / (z250_val - z500_val)
    N2 = (g / theta_bar) * dtheta_dz
    
    # Richardson
    Ri = (N2 / shear2).where(np.isfinite(N2) & np.isfinite(shear2))
    Ri.name = "Ri_250_500"

    print(f"DEBUG {modelo}: Ri min={float(Ri.min()):.3e}, max={float(Ri.max()):.3e}, "
      f"%Ri<5={float((Ri<5).sum()/Ri.size*100):.2f}%")
    
    # Guardar para multimodelo (solo turbulencia potencial)
    Ri_daily_all_models.append(Ri.assign_coords(modelo=modelo))

    # Frecuencias de inestabilidad
    valid_days = xr.where(np.isfinite(Ri), 1.0, np.nan)
    Ri_flag_5 = xr.where(Ri < 5.0, 1.0, 0.0)

    Ri_freq_5 = (Ri_flag_5.sum("time") / valid_days.sum("time")).rename("Ri_freq_lt_1")

    Ri_freq_5_models.append(Ri_freq_5.expand_dims(modelo=[modelo]))

# FIGURAS CONJUNTAS DE FRECUENCIAS (todos los modelos)

import matplotlib.colors as mcolors
from matplotlib.colors import LogNorm

levels = [0, 5, 10, 15, 20, 25, 30]
# colors = ["#f7f7f7", "#fee090", "#fdae61", "#d73027", "#abd9e9", "#74add1", "#4575b4","#313695"]
# cmap = mcolors.ListedColormap(colors)
# norm = mcolors.BoundaryNorm(levels, ncolors=cmap.N, extend="both")
cmap = plt.get_cmap("coolwarm")
norm = mcolors.BoundaryNorm(levels, ncolors=cmap.N, extend="max")

for Ri_list, thr, fname in [
    (Ri_freq_5_models, "Ri < 5", "Ri_freq5_todos.png")
]:
    nmod = len(Ri_list)
    ncols = 2
    nrows = int(np.ceil(nmod / ncols))
    fig, axes = plt.subplots(
        nrows, ncols, figsize=(4*ncols, 3*nrows),
        subplot_kw={'projection': ccrs.PlateCarree()}
    )
    axes = axes.flatten()

    for i, (ax, ri_freq) in enumerate(zip(axes, Ri_list)):
        freq_pct = ri_freq.squeeze() * 100.0
        freq_pct.plot(
            ax=ax, transform=ccrs.PlateCarree(),
            cmap=cmap, norm=norm,
            cbar_kwargs={
                "label": "% del tiempo DJF",
                "orientation": "horizontal",
                "pad": 0.1, "shrink": 0.7,
                "ticks": levels
            }
        )
        ax.coastlines(); ax.add_feature(cfeature.BORDERS, linewidth=0.5)
        ax.set_extent([280, 352.5, 35, 69], crs=ccrs.PlateCarree())
        ax.set_title(modelos[i])

    # Ocultar subplots vacíos
    for j in range(i+1, len(axes)):
        fig.delaxes(axes[j])

    plt.suptitle(f"Frecuencia DJF de {thr}\n({escenario}, 2030–2100)", fontsize=12)
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.savefig(os.path.join(base_out, fname), dpi=300, bbox_inches="tight")
    plt.close(fig)

print("Figuras conjuntas de frecuencias Ri<5 guardadas.")

# FRECUENCIA MULTIMODELO Y CATEGORÍAS DE ESTABILIDAD

if Ri_freq_5_models:
    Ri_freq_5_all = xr.concat(Ri_freq_5_models, dim="modelo")

    Ri_freq_5_mean = Ri_freq_5_all.mean("modelo", skipna=True)

    ####################################################################################
    
    # MEDIA MULTIMODELO DE FRECUENCIA Ri < 5
    
    fig = plt.figure(figsize=(7, 4))
    ax = plt.axes(projection=ccrs.PlateCarree())
    
    # Convertir a porcentaje
    Ri_freq_5_mean_pct = Ri_freq_5_mean * 100.0
    
    # Definir niveles y paleta
    levels = np.arange(0, 31, 5)
    cmap = plt.get_cmap("coolwarm")
    norm = mcolors.BoundaryNorm(levels, ncolors=cmap.N, extend="max")
    
    # Dibujar el mapa
    Ri_freq_5_mean_pct.plot(
        ax=ax, transform=ccrs.PlateCarree(),
        cmap=cmap, norm=norm,
        cbar_kwargs={
            "label": "% del tiempo DJF con Ri < 5 (2030–2100)",
            "orientation": "horizontal",
            "pad": 0.1, "shrink": 0.7,
            "ticks": levels
        }
    )
    
    # Estética del mapa
    ax.coastlines(linewidth=0.7)
    ax.add_feature(cfeature.BORDERS, linewidth=0.4)
    ax.set_extent([280, 352.5, 35, 69], crs=ccrs.PlateCarree())

    # Etiquetas de latitud/longitud
    gl = ax.gridlines(
        crs=ccrs.PlateCarree(),
        draw_labels=True,
        linewidth=0.5, color='gray', alpha=0.5, linestyle='--'
    )
    
    gl.top_labels = False         # sin etiquetas arriba
    gl.right_labels = False       # sin etiquetas a la derecha
    gl.xlabel_style = {"size": 8}
    gl.ylabel_style = {"size": 8}
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER

    # Añadir la ruta DUB–JFK
    puntos = geod.npts(lon_DUB, lat_DUB, lon_JFK, lat_JFK, 100)
    lons = [lon_DUB] + [p[0] for p in puntos] + [lon_JFK]
    lats = [lat_DUB] + [p[1] for p in puntos] + [lat_JFK]
    ax.plot(lons, lats, transform=ccrs.Geodetic(),
            color="black", linewidth=0.8, linestyle="-")

    plt.title(f"Frecuencia media multimodelo de Ri < 5\n{escenario} (DJF 2030–2100)", fontsize=10)
    plt.tight_layout()
    plt.savefig(os.path.join(base_out, "Ri_freq5_media_multimodelo.png"),
                dpi=300, bbox_inches="tight")
    plt.close(fig)
    
    print("✔ Mapa multimodelo de frecuencia Ri<5 guardado.")

    # ARMONIZAR CALENDARIOS ANTES DE CONCATENAR Ri DIARIO
    
    Ri_daily_all_models_fixed = []
    
    for ds in Ri_daily_all_models:
        # Si existe coordenada "time", la convertimos a índice numérico de días
        if "time" in ds.coords:
            # Guardar una copia del tiempo original (opcional, para trazabilidad)
            ds["time_original"] = ds["time"]
    
            ds_fixed = ds.assign_coords(day=("time", np.arange(ds.sizes["time"])))

            # Sustituir "time" por "day" como dimensión principal
            ds_fixed = ds_fixed.swap_dims({"time": "day"}).drop_vars("time")
        else:
            ds_fixed = ds  # por si acaso algún modelo ya no tiene 'time'
    
        Ri_daily_all_models_fixed.append(ds_fixed)
    
    # Ahora concatenamos sin errores de calendario
    Ri_daily_all = xr.concat(Ri_daily_all_models_fixed, dim="modelo")
    
    # Si prefieres seguir usando el nombre 'time', puedes renombrar:
    Ri_daily_all = Ri_daily_all.rename({"day": "time"})

    Ri_mean = Ri_daily_all.mean("time").mean("modelo")

    # Clasificación simple por estabilidad
    Ri_categorias = xr.full_like(Ri_mean, np.nan)
    Ri_categorias = xr.where(Ri_mean >= 10, 1, Ri_categorias)     # Estable
    Ri_categorias = xr.where((Ri_mean < 10) & (Ri_mean >= 4), 2, Ri_categorias)  # Est. débil
    Ri_categorias = xr.where((Ri_mean < 4) & (Ri_mean >= 1), 3, Ri_categorias)   # Transición
    Ri_categorias = xr.where((Ri_mean < 1) & (Ri_mean >= 0.25), 4, Ri_categorias) # Inestable
    Ri_categorias = xr.where(Ri_mean < 0.25, 5, Ri_categorias)   # Muy inestable

    labels_Ri = ["Estable", "Est. débil", "Transición", "Inestable", "Muy inestable"]
    cmap = plt.get_cmap("RdYlBu_r", len(labels_Ri))
    
# MEDIA MULTIMODELO DE Ri (no frecuencia)
#     - media en el tiempo por modelo
#     - luego media entre modelos

# Armonizar calendarios antes de concatenar Ri diario, en caso de no haberse hecho
Ri_daily_all_models_fixed = []
for ds in Ri_daily_all_models:
    if "time" in ds.coords:
        ds_fixed = ds.assign_coords(day=("time", np.arange(ds.sizes["time"])))
        ds_fixed = ds_fixed.swap_dims({"time": "day"}).drop_vars("time")
    else:
        ds_fixed = ds
    Ri_daily_all_models_fixed.append(ds_fixed)

# Concatenar sin conflictos de calendario
Ri_daily_all = xr.concat(Ri_daily_all_models_fixed, dim="modelo")
Ri_daily_all = Ri_daily_all.rename({"day": "time"})

# Media temporal por modelo y media multimodelo
Ri_mean_time = Ri_daily_all.mean("time", skipna=True)         # Ri medio DJF por modelo
Ri_mean_mm   = Ri_mean_time.mean("modelo", skipna=True)       # Ri medio DJF multimodelo

# Mapa de Ri medio multimodelo
fig = plt.figure(figsize=(7, 4))
ax = plt.axes(projection=ccrs.PlateCarree())

# Escala basada en percentiles (robusta a outliers)
vmin = float(Ri_mean_mm.quantile(0.02))
vmax = float(Ri_mean_mm.quantile(0.98))
if not np.isfinite(vmin) or not np.isfinite(vmax) or vmin >= vmax:
    vmin, vmax = float(Ri_mean_mm.min()), float(Ri_mean_mm.max())

levels = np.linspace(vmin, vmax, 9)

# Paleta clara y cálida-fría suave
cmap = plt.cm.get_cmap("YlGnBu", 256).copy()
cmap.set_under("#dddddd")  # valores muy bajos quedan blanquecinos

norm = mcolors.BoundaryNorm(levels, ncolors=cmap.N, extend="min")

imRi = Ri_mean_mm.plot(
    ax=ax, transform=ccrs.PlateCarree(),
    cmap=cmap, norm=norm,
    cbar_kwargs={
        "label": "Ri medio 250–500 hPa (DJF 2030–2100)",
        "orientation": "horizontal", "pad": 0.1, "shrink": 0.7,
        "ticks": levels
    }
)

ax.coastlines(linewidth=0.7)
ax.add_feature(cfeature.BORDERS, linewidth=0.4)
ax.set_extent([280, 352.5, 35, 69], crs=ccrs.PlateCarree())

# Rejilla con etiquetas
gl = ax.gridlines(
    crs=ccrs.PlateCarree(), draw_labels=True,
    linewidth=0.5, color='gray', alpha=0.5, linestyle='--'
)
gl.top_labels = False
gl.right_labels = False
gl.xlabel_style = {"size": 8}
gl.ylabel_style = {"size": 8}
gl.xformatter = LONGITUDE_FORMATTER
gl.yformatter = LATITUDE_FORMATTER

# Ruta DUB–JFK
puntos = geod.npts(lon_DUB, lat_DUB, lon_JFK, lat_JFK, 100)
lons = [lon_DUB] + [p[0] for p in puntos] + [lon_JFK]
lats = [lat_DUB] + [p[1] for p in puntos] + [lat_JFK]
ax.plot(lons, lats, transform=ccrs.Geodetic(),
        color="black", linewidth=0.8, linestyle="-")

plt.title(f"Ri medio multimodelo (250–500 hPa)\n{escenario} – DJF 2030–2100", fontsize=10)
plt.tight_layout()
plt.savefig(os.path.join(base_out, "Ri_media_multimodelo.png"),
            dpi=300, bbox_inches="tight")
plt.close(fig)

print("Mapa multimodelo de Ri medio guardado.")

# FRECUENCIA DJF de (Ri_multimodelo_diario < 5)
#     (primero media entre modelos por día, luego % de días)

print("Calculando frecuencia DJF de Ri<5 a partir del Ri multimodelo diario...")

# Armonizar calendarios: time -> day por cada modelo y concatenar
Ri_daily_all_models_fixed = []
for ds in Ri_daily_all_models:
    if "time" in ds.coords:
        ds_fixed = ds.assign_coords(day=("time", np.arange(ds.sizes["time"])))
        ds_fixed = ds_fixed.swap_dims({"time": "day"}).drop_vars("time")
    else:
        ds_fixed = ds
    Ri_daily_all_models_fixed.append(ds_fixed)

# Concatenar por modelo y (opcional) volver a llamar "time" al eje
Ri_daily_all = xr.concat(Ri_daily_all_models_fixed, dim="modelo")
Ri_daily_all = Ri_daily_all.rename({"day": "time"})

# Multimodelo diario (media entre modelos para cada día)
Ri_mm_diario = Ri_daily_all.mean("modelo", skipna=True)

# % de días con Ri_multimodelo_diario < 5
Ri_mm_freq5_pct = ( (Ri_mm_diario < 5).mean("time") * 100 ).rename("pct_days_RiMM_lt5")

# Graficar (log adaptativo y ceros como punteado tenue)
fig = plt.figure(figsize=(7, 4))
ax = plt.axes(projection=ccrs.PlateCarree())

freq = Ri_mm_freq5_pct  # en %
p98 = float(freq.quantile(0.98))
use_log = np.isfinite(p98) and p98 <= 10.0

if use_log:
    eps = 1e-2  # 0.01 %
    data_plot = freq.clip(min=eps)
    vmin = max(eps, float(data_plot.quantile(0.05)))
    vmax = float(data_plot.quantile(0.98))
    if not np.isfinite(vmax) or vmax <= vmin:
        vmax = max(vmin * 10, 1.0)

    norm = mcolors.LogNorm(vmin=vmin, vmax=vmax)
    cmap = plt.cm.get_cmap("YlOrBr", 256)
    cmap.set_under("whitesmoke")

    im4 = data_plot.plot(
        ax=ax, transform=ccrs.PlateCarree(),
        cmap=cmap, norm=norm, add_colorbar=False
    )

    # barra de color con ticks en %
    from matplotlib.ticker import LogLocator, FuncFormatter
    cbar = plt.colorbar(
        plt.cm.ScalarMappable(norm=norm, cmap=cmap),
        ax=ax, orientation="horizontal", pad=0.10, shrink=0.7
    )
    # posibles ticks (se filtran por el rango)
    tick_candidates = np.array([0.01, 0.03, 0.1, 0.3, 1.0, 3.0, 10.0, 30.0])
    ticks = [t for t in tick_candidates if vmin <= t <= vmax]
    if len(ticks) >= 2:
        cbar.set_ticks(ticks)

    cbar.formatter = FuncFormatter(lambda x, pos: f"{x:.2g} %")
    cbar.set_label("% del tiempo DJF con Ri (media diaria multimodelo) < 5 (2030–2100)")
    cbar.update_ticks()
else:
    # escala lineal adaptativa
    vmin = 0.0
    vmax_raw = p98 if np.isfinite(p98) and p98 > 0 else float(freq.max())
    vmax = float(np.ceil(max(1.0, vmax_raw) / 2.5) * 2.5)
    levels = np.linspace(vmin, vmax, int(max(2, vmax // 2.5) + 1))

    cmap = plt.get_cmap("YlOrBr")
    norm = mcolors.BoundaryNorm(levels, ncolors=cmap.N, extend="max")

    im5 = freq.plot(
        ax=ax, transform=ccrs.PlateCarree(),
        cmap=cmap, norm=norm,
        cbar_kwargs={
            "label": "% del tiempo DJF con Ri (media diaria multimodelo) < 5 (2030–2100)",
            "orientation": "horizontal", "pad": 0.10, "shrink": 0.7,
            "ticks": levels
        }
    )

# Ceros como puntos suaves (en lugar de trama encima del mapa)
try:
    zeros = (freq <= 0)
    if zeros.any():
        lon = freq["lon"].values
        lat = freq["lat"].values
        zz = zeros.values
        # coordenadas de puntos con cero
        jj, ii = np.where(zz)
        if jj.size:
            ax.scatter(lon[ii], lat[jj], s=3, marker=".", color="k", alpha=0.20,
                       transform=ccrs.PlateCarree(), zorder=3)
except Exception:
    pass

# Estética del mapa
ax.coastlines(linewidth=0.7)
ax.add_feature(cfeature.BORDERS, linewidth=0.4)
ax.set_extent([280, 352.5, 35, 69], crs=ccrs.PlateCarree())

# Gridlines
gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                  linewidth=0.5, color='gray', alpha=0.5, linestyle='--')
gl.top_labels = False
gl.right_labels = False
gl.xformatter = LONGITUDE_FORMATTER
gl.yformatter = LATITUDE_FORMATTER
gl.xlabel_style = {"size": 8}
gl.ylabel_style = {"size": 8}

# Ruta DUB–JFK
puntos = geod.npts(lon_DUB, lat_DUB, lon_JFK, lat_JFK, 100)
lons = [lon_DUB] + [p[0] for p in puntos] + [lon_JFK]
lats = [lat_DUB] + [p[1] for p in puntos] + [lat_JFK]
ax.plot(lons, lats, transform=ccrs.Geodetic(),
        color="black", linewidth=0.8, linestyle="-")

plt.title(f"Frecuencia DJF de Ri multimodelo diario < 5\n{escenario} (2030–2100)", fontsize=10)
plt.tight_layout()
plt.savefig(os.path.join(base_out, "Ri_freq5_media_sobre_media_diaria_multimodelo.png"),
            dpi=300, bbox_inches="tight")
plt.close(fig)

# GUARDAR RESULTADOS MULTIMODELO EN NetCDF

# Guardar la media multimodelo de Ri
Ri_mean_mm.to_netcdf(os.path.join(base_out, f"Ri_mean_multimodelo_{escenario}.nc"))
print(f"Archivo guardado: Ri_mean_multimodelo_{escenario}.nc")

# Guardar la frecuencia multimodelo (Ri < 5)
Ri_freq_5_mean_pct.to_netcdf(os.path.join(base_out, f"Ri_freq5_multimodelo_{escenario}.nc"))
print(f"Archivo guardado: Ri_freq5_multimodelo_{escenario}.nc")

print("Cálculo y gráfico de categorías multimodelo de Ri completado.")

In [ ]:
# MULTIMODELO (media de tendencias de TI1)
from matplotlib.ticker import LogLocator, FormatStrFormatter

shear_all = xr.concat(shear_slopes, dim="modelo")
ti1_all   = xr.concat(ti1_slopes, dim="modelo")

shear_mean = shear_all.mean("modelo", skipna=True)
ti1_mean   = ti1_all.mean("modelo", skipna=True)

# Graficar tendencias de TI1 multimodelo
fig = plt.figure(figsize=(7, 4))
ax = plt.axes(projection=ccrs.PlateCarree())

# Escala simétrica basada en percentiles
vmin = float((ti1_mean * 1e7).quantile(0.02))
vmax = float((ti1_mean * 1e7).quantile(0.98))
lim = max(abs(vmin), abs(vmax))
norm = mcolors.TwoSlopeNorm(vmin=-lim, vcenter=0, vmax=lim)

im_ti1 = (ti1_mean * 1e7).plot(
    ax=ax, transform=ccrs.PlateCarree(),
    cmap="coolwarm", norm=norm,
    cbar_kwargs={"label": "Tendencia TI1 multimodelo (×10⁻⁷ s⁻² por década)",
                 "orientation":"horizontal", "pad":0.1, "shrink":0.7}
)

cs = ax.contour(
    ti1_mean * 1e7,
    levels=np.linspace(-lim, lim, 8),   # ya en las mismas unidades del plot (×10⁻⁷)
    colors="k", linewidths=0.3,
    transform=ccrs.PlateCarree()
)

ax.clabel(cs, fmt="%.1f", fontsize=6, inline=True)

ax.coastlines(); ax.add_feature(cfeature.BORDERS, linewidth=0.5)
ax.set_extent([280, 352.5, 35, 69], crs=ccrs.PlateCarree())

# Añadir la ruta DUB–JFK
puntos = geod.npts(lon_DUB, lat_DUB, lon_JFK, lat_JFK, 100)
lons = [lon_DUB] + [p[0] for p in puntos] + [lon_JFK]
lats = [lat_DUB] + [p[1] for p in puntos] + [lat_JFK]
ax.plot(lons, lats, transform=ccrs.Geodetic(),
        color="black", linewidth=0.8, linestyle="-")

# Añadir etiquetas de lat/lon
gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                  linewidth=0.5, color='gray', alpha=0.5, linestyle='--')

# Configuración de etiquetas
gl.top_labels = False     # Quita etiquetas arriba
gl.right_labels = False   # Quita etiquetas a la derecha
gl.xformatter = LONGITUDE_FORMATTER
gl.yformatter = LATITUDE_FORMATTER
gl.xlabel_style = {"size": 8}
gl.ylabel_style = {"size": 8}

plt.title(f"Tendencia multimodelo TI1 (250–500 hPa)\nDJF 2030–2100 ({escenario})", fontsize=9)
plt.savefig(os.path.join(base_out, "TI1_multimodelo_tendencias.png"), dpi=300, bbox_inches="tight")
plt.close(fig)

# MULTIMODELO – CLIMATOLOGÍA MEDIA DE TI1

if ti1_mean_models:
    print("\nCalculando TI1 climatológico multimodelo...")

    ti1_clim_all = xr.concat(ti1_mean_models, dim="modelo")
    ti1_clim_mean = ti1_clim_all.mean("modelo", skipna=True)
    
    # ARMONIZAR CALENDARIOS ANTES DE CONCATENAR TI1 DIARIO
    ti1_daily_all_models_fixed = []
    
    for ds in ti1_daily_all_models:
        if "time" in ds.coords:
            ds["time_original"] = ds["time"]
            ds_fixed = ds.assign_coords(day=("time", np.arange(ds.sizes["time"])))
            ds_fixed = ds_fixed.swap_dims({"time": "day"}).drop_vars("time")
        else:
            ds_fixed = ds
        ti1_daily_all_models_fixed.append(ds_fixed)
    
    # Ahora podemos concatenar sin errores de calendario
    ti1_djf_all = xr.concat(ti1_daily_all_models_fixed, dim="modelo")
    
    #(Opcional) volver a llamar "time" al eje
    ti1_djf_all = ti1_djf_all.rename({"day": "time"})
    
    # Aplanar todas las dimensiones relevantes
    ti1_flat = ti1_djf_all.stack(points=("modelo", "time", "lat", "lon"))
    
    # Elimina posibles NaNs
    ti1_flat = ti1_flat.where(np.isfinite(ti1_flat), drop=True)
  
    ## CLASIFICACIÓN DE TI₁ USANDO UMBRALES FIJOS
    
    print("Clasificando intensidad de turbulencia según umbrales fijos (basados en SSP245)...")
    
    # Clasificar según los umbrales cargados o calculados antes
    ti1_cats_daily = clasificar_por_umbral(ti1_djf_all, thr_ti1)
    
    # Calcular frecuencia (%) de cada categoría
    ti1_freq_lig = (ti1_cats_daily == 1).mean(("modelo", "time")) * 100
    ti1_freq_mod = (ti1_cats_daily == 2).mean(("modelo", "time")) * 100
    ti1_freq_sev = (ti1_cats_daily == 3).mean(("modelo", "time")) * 100
    
    # Guardar resultados medios
    ti1_freq_total = xr.Dataset({
        "Ligera - Ligera/Moderada (%)":   ti1_freq_lig,
        "Moderada - Moderada/Severa (%)": ti1_freq_mod,
        "Severa (%)":                     ti1_freq_sev
    })
    
    for nombre, data in ti1_freq_total.items():
        fig = plt.figure(figsize=(7, 4))
        ax = plt.axes(projection=ccrs.PlateCarree())
        im = data.plot(
            ax=ax, transform=ccrs.PlateCarree(),
            cmap=cmap_ti1, vmin=0, vmax=float(data.max()),
            cbar_kwargs={"label": f"% de días DJF con turbulencia {nombre.split()[0].lower()} (TI1)",
                         "orientation":"horizontal", "pad":0.1, "shrink":0.7}
        )
        ax.coastlines(); ax.add_feature(cfeature.BORDERS, linewidth=0.5)
        ax.set_extent([280, 352.5, 35, 69], crs=ccrs.PlateCarree())
    
        plt.title(f"Frecuencia multimodelo de turbulencia {nombre.split()[0].lower()} (TI1)\nDJF 2030–2100 ({escenario})", fontsize=9)
        plt.savefig(os.path.join(base_out, f"TI1_frecuencia_{nombre.split()[0].lower()}_multimodelo.png"),
                    dpi=300, bbox_inches="tight")
        plt.close(fig)

    
    print("Frecuencia de turbulencia severa (TI₁) graficada y guardada.")

    print(f"Total de valores diarios en la muestra: {ti1_flat.size:,}")

    # Graficar TI1 medio multimodelo
    fig = plt.figure(figsize=(7, 4))
    ax = plt.axes(projection=ccrs.PlateCarree())
    vmax = float((ti1_clim_mean * 1e7).quantile(0.98))
    norm = mcolors.Normalize(vmin=0, vmax=vmax)

    im_clim = (ti1_clim_mean * 1e7).plot(
        ax=ax, transform=ccrs.PlateCarree(),
        cmap=cmap_ti1, norm=mcolors.Normalize(vmin=0, vmax=vmax),
        cbar_kwargs={"label": "Media multimodelo de TI1 en DJF (×10⁻⁷ s⁻²)",
                     "orientation":"horizontal", "pad":0.1, "shrink":0.7}
    )
    
    # Mejorar espaciado del texto de la barra de color
    cbar = im_clim.colorbar
    cbar.ax.tick_params(labelsize=7, pad=3)
    cbar.ax.xaxis.get_offset_text().set_position((0, -0.5))
    
    cs = ax.contour(
        ti1_clim_mean * 1e7,
        levels=np.linspace(0, vmax, 8),   # mismo sistema de unidades (×10⁻⁷)
        colors='k', linewidths=0.3,
        transform=ccrs.PlateCarree()
    )

    ax.clabel(cs, fmt="%.1f", fontsize=6, inline=True)

    # Añadir la ruta DUB–JFK
    puntos = geod.npts(lon_DUB, lat_DUB, lon_JFK, lat_JFK, 100)
    lons = [lon_DUB] + [p[0] for p in puntos] + [lon_JFK]
    lats = [lat_DUB] + [p[1] for p in puntos] + [lat_JFK]
    ax.plot(lons, lats, transform=ccrs.Geodetic(),
            color="black", linewidth=0.8, linestyle="-")

    ax.coastlines(); ax.add_feature(cfeature.BORDERS, linewidth=0.5)
    ax.set_extent([280, 352.5, 35, 69], crs=ccrs.PlateCarree())

    # Añadir etiquetas de latitud y longitud
    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                      linewidth=0.5, color='gray', alpha=0.5, linestyle='--')
    
    # Configuración de etiquetas
    gl.top_labels = False       # Quitar etiquetas superiores
    gl.right_labels = False     # Quitar etiquetas a la derecha
    gl.xlabel_style = {"size": 8}
    gl.ylabel_style = {"size": 8}
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER

    plt.title(f"TI1 medio multimodelo (250–500 hPa)\nDJF 2030–2100 ({escenario})", fontsize=9)
    plt.tight_layout()  # mejora el espaciado general
    plt.savefig(os.path.join(base_out, "TI1_medio_multimodelo.png"),
                dpi=300, bbox_inches="tight")
    plt.close(fig)
    print("Mapa multimodelo de medias de TI1 guardado.")

# MAPA FINAL TENDENCIAS DE CIZALLADURA con zg
# Combina las tendencias de todos los modelos
VWS_all = xr.concat(shear_slopes, dim="modelo")
VWS_mean = VWS_all.mean("modelo", skipna=True)

# Gráfico
fig = plt.figure(figsize=(7, 4))
ax = plt.axes(projection=ccrs.PlateCarree())

# Escala simétrica común
lim = max(abs(float(VWS_mean.min())), abs(float(VWS_mean.max())))
norm = mcolors.TwoSlopeNorm(vmin=-lim, vcenter=0, vmax=lim)

# Tendencias (no se escalan)
im = VWS_mean.plot(
    ax=ax, transform=ccrs.PlateCarree(),
    cmap="RdBu_r", norm=norm,
    cbar_kwargs={
        "label": "Tendencia multimodelo de VWS (s⁻¹ por década)",
        "orientation": "horizontal", "pad": 0.15, "shrink": 0.7
    }
)

# Mover el “1e−5” más abajo para que no choque con el texto
cbar = im.colorbar
cbar.ax.xaxis.get_offset_text().set_position((0, -0.5))
cbar.ax.tick_params(labelsize=7, pad=3)

cs = ax.contour(
    VWS_mean,
    levels=np.linspace(-lim, lim, 9),
    colors="black", linewidths=0.3,
    transform=ccrs.PlateCarree()
)
ax.clabel(cs, fmt="%.1e", fontsize=6, inline=True)

ax.coastlines()
ax.add_feature(cfeature.BORDERS, linewidth=0.5)
ax.set_extent([280, 352.5, 35, 69], crs=ccrs.PlateCarree())

# Añadir etiquetas de lat/lon
gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                  linewidth=0.5, color='gray', alpha=0.5, linestyle='--')
gl.top_labels = False
gl.right_labels = False
gl.xformatter = LONGITUDE_FORMATTER
gl.yformatter = LATITUDE_FORMATTER
gl.xlabel_style = {"size": 8}
gl.ylabel_style = {"size": 8}

# Añadir la ruta DUB–JFK
puntos = geod.npts(lon_DUB, lat_DUB, lon_JFK, lat_JFK, 100)
lons = [lon_DUB] + [p[0] for p in puntos] + [lon_JFK]
lats = [lat_DUB] + [p[1] for p in puntos] + [lat_JFK]
ax.plot(lons, lats, transform=ccrs.Geodetic(),
        color="black", linewidth=0.8, linestyle="-")

plt.title(f"Tendencia multimodelo de cizalladura 250–500 hPa (VWS)\nDJF 2030–2100 ({escenario})", fontsize=9)

os.makedirs(base_out, exist_ok=True)
plt.savefig(os.path.join(base_out, "VWS_tendencia_multimodelo.png"),
            bbox_inches="tight", dpi=300)
plt.close(fig)

print("Figura multimodelo de tendencia VWS guardada.")

# MAPA MEDIA MULTIMODELO DE CIZALLADURA con zg (VWS)
print("Creando mapa multimodelo de VWS medio (DJF 2030–2100)...")

# Combina las medias DJF de todos los modelos
VWS_all_mean = xr.concat(VWS_mean_models, dim="modelo")
VWS_multimodel_mean = VWS_all_mean.mean("modelo", skipna=True)

# Gráfico
fig = plt.figure(figsize=(7, 4))
ax = plt.axes(projection=ccrs.PlateCarree())

vmin = float(VWS_multimodel_mean.quantile(0.05)) * 1e3
vmax = float(VWS_multimodel_mean.quantile(0.90)) * 1e3
norm = mcolors.Normalize(vmin=0, vmax=vmax)

im = (VWS_multimodel_mean * 1e3).plot(
    ax=ax, transform=ccrs.PlateCarree(),
    cmap="YlOrRd", norm=mcolors.Normalize(vmin=vmin, vmax=vmax),
    cbar_kwargs={
        "label": "VWS medio DJF (×10⁻³ s⁻¹, 2030–2100)",
        "orientation": "horizontal", "pad": 0.1, "shrink": 0.7
    }
)

ax.coastlines()
ax.add_feature(cfeature.BORDERS, linewidth=0.5)
ax.set_extent([280, 352.5, 35, 69], crs=ccrs.PlateCarree())

# Etiquetas de lat/lon
gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                  linewidth=0.5, color='gray', alpha=0.5, linestyle='--')
gl.top_labels = False
gl.right_labels = False
gl.xformatter = LONGITUDE_FORMATTER
gl.yformatter = LATITUDE_FORMATTER
gl.xlabel_style = {"size": 8}
gl.ylabel_style = {"size": 8}

# Ruta DUB–JFK
puntos = geod.npts(lon_DUB, lat_DUB, lon_JFK, lat_JFK, 100)
lons = [lon_DUB] + [p[0] for p in puntos] + [lon_JFK]
lats = [lat_DUB] + [p[1] for p in puntos] + [lat_JFK]
ax.plot(lons, lats, transform=ccrs.Geodetic(),
        color="black", linewidth=0.8, linestyle="-")

plt.title("VWS medio multimodelo 250–500 hPa\nDJF 2030–2100", fontsize=9)

os.makedirs(base_out, exist_ok=True)
plt.savefig(os.path.join(base_out, "VWS_media_multimodelo.png"),
            bbox_inches="tight", dpi=300)
plt.close(fig)

print("✔ Figura multimodelo de VWS medio guardada.")

# CLASIFICACIÓN DE VWS USANDO UMBRALES FIJOS (desde SSP245)
print("\nUsando umbrales fijos de VWS (definidos a partir de SSP245)...")

# Armonizar calendarios antes de concatenar VWS diario
VWS_daily_all_models_fixed = []
for ds in VWS_daily_all_models:
    if "time" in ds.coords:
        ds["time_original"] = ds["time"]
        ds_fixed = ds.assign_coords(day=("time", np.arange(ds.sizes["time"])))
        ds_fixed = ds_fixed.swap_dims({"time": "day"}).drop_vars("time")
    else:
        ds_fixed = ds
    VWS_daily_all_models_fixed.append(ds_fixed)

# Concatenar sin errores de calendario
VWS_daily_all = xr.concat(VWS_daily_all_models_fixed, dim="modelo")
VWS_daily_all = VWS_daily_all.rename({"day": "time"})

# CLASIFICACIÓN DE VWS USANDO UMBRALES FIJOS
VWS_cats_daily = clasificar_por_umbral(VWS_daily_all, thr_vws)

# Frecuencia de días con cizalladura moderada o más fuerte
VWS_freq_mod = ((VWS_cats_daily >= 2) & (VWS_cats_daily <= 3)).mean(("modelo", "time")) * 100
VWS_freq_lig_mod = ((VWS_cats_daily >= 1) & (VWS_cats_daily <= 2)).mean(("modelo", "time")) * 100

print("Clasificación de VWS completada con umbrales fijos.")
print(f"Total de valores considerados: {VWS_daily_all.size:,}")

# FRECUENCIAS POR CATEGORÍA (TI₁ y VWS)
print("Creando gráfico conjunto de frecuencias por categoría (TI₁ y VWS)...")

# TI₁
categorias_ti1 = {
    "Ligera - Ligera/Moderada": 1,
    "Moderada - Moderada/Severa": 2,
    "Severa": 3}

# VWS
categorias_vws = {
    "Ligera - Ligera/Moderada": 1,
    "Moderada - Moderada/Severa": 2,
    "Severa": 3}
    
ti1_freq_por_cat = {
    nombre: (ti1_cats_daily == val).mean(("modelo", "time")) * 100
    for nombre, val in categorias_ti1.items()
}
VWS_freq_por_cat = {
    nombre: (VWS_cats_daily == val).mean(("modelo", "time")) * 100
    for nombre, val in categorias_vws.items()
}


# Preparar datos para gráfico
labels = ["Ligera", "Moderada", "Severa"]
ti1_vals = [
    float(ti1_freq_por_cat["Ligera - Ligera/Moderada"].mean().values),
    float(ti1_freq_por_cat["Moderada - Moderada/Severa"].mean().values),
    float(ti1_freq_por_cat["Severa"].mean().values)
]
vws_vals = [
    float(VWS_freq_por_cat["Ligera - Ligera/Moderada"].mean().values),
    float(VWS_freq_por_cat["Moderada - Moderada/Severa"].mean().values),
    float(VWS_freq_por_cat["Severa"].mean().values)
]

x = np.arange(len(labels))
width = 0.35

# Gráfico
fig, ax = plt.subplots(figsize=(6, 4))
bars1 = ax.bar(x - width/2, ti1_vals, width, label="TI₁", color="#fdae61")  # cálido
bars2 = ax.bar(x + width/2, vws_vals, width, label="VWS", color="#4575b4")  # frío

ax.set_ylabel("% de días DJF 2030–2100")
ax.set_title(f"Frecuencia multimodelo por tipo de turbulencia\n(TI₁ y VWS) – {escenario}")
ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.legend()

# Ajustar eje y etiquetas
max_val = max(max(ti1_vals), max(vws_vals))
ax.set_ylim(0, max_val * 1.15)

for bars in [bars1, bars2]:
    for bar in bars:
        h = bar.get_height()
        ax.annotate(f"{h:.2f}%", xy=(bar.get_x() + bar.get_width()/2, h),
                    xytext=(0,3), textcoords="offset points",
                    ha="center", va="bottom", fontsize=8)

plt.tight_layout()
plt.savefig(os.path.join(base_out, f"frecuencia_turbulencias_por_categoria_TI1_VWS_{escenario}.png"),
            dpi=300, bbox_inches="tight")
plt.close(fig)

print("Gráfico de barras por categoría (Ligera–Moderada–Severa) guardado.")

# FIGURA COMPARATIVA DE MAPAS POR CATEGORÍA (TI₁ vs VWS)
import matplotlib.colors as mcolors
from matplotlib.ticker import LogLocator, LogFormatterMathtext

print("Creando figura comparativa (TI₁ vs VWS) con leyendas separadas al pie...")

labels = ["Ligera - Ligera/Moderada", "Moderada - Moderada/Severa", "Severa"]

# Figura más compacta
fig, axes = plt.subplots(len(labels), 2, figsize=(8.5, 7),
                         subplot_kw={'projection': ccrs.PlateCarree()})
plt.subplots_adjust(hspace=0.15)

# Colormaps
cmap_ti1 = plt.cm.get_cmap("YlOrBr", 256)
cmap_vws = plt.cm.get_cmap("YlOrRd", 256) # YlGnBu_r
cmap_ti1.set_under("whitesmoke")
cmap_vws.set_under("whitesmoke")

# Escalas
# Lineales (Ligera)
vmax_ti1_light = float(ti1_freq_por_cat["Ligera - Ligera/Moderada"].quantile(0.98))
vmax_vws_light = float(VWS_freq_por_cat["Ligera - Ligera/Moderada"].quantile(0.98))
norm_ti1_light = mcolors.Normalize(vmin=0, vmax=vmax_ti1_light)
norm_vws_light = mcolors.Normalize(vmin=0, vmax=vmax_vws_light)

# Logarítmicas (Moderada/Severa)
ti1_modsev = xr.concat(
    [ti1_freq_por_cat["Moderada - Moderada/Severa"], ti1_freq_por_cat["Severa"]],
    dim="cat"
)
vws_modsev = xr.concat(
    [VWS_freq_por_cat["Moderada - Moderada/Severa"], VWS_freq_por_cat["Severa"]],
    dim="cat"
)

vmin_ti1_ms = 1e-3
vmin_vws_ms = 1e-3
vmax_ti1_ms = float(ti1_modsev.quantile(0.95))
vmax_vws_ms = float(vws_modsev.quantile(0.95))

# Evitar ceros
ti1_modsev = ti1_modsev.clip(min=vmin_ti1_ms)
vws_modsev = vws_modsev.clip(min=vmin_vws_ms)

norm_ti1_ms = mcolors.LogNorm(vmin=vmin_ti1_ms, vmax=vmax_ti1_ms)
norm_vws_ms = mcolors.LogNorm(vmin=vmin_vws_ms, vmax=vmax_vws_ms)

# Mapas
for i, cat in enumerate(labels):
    if "Ligera" in cat:
        norm_ti1_use, norm_vws_use = norm_ti1_light, norm_vws_light
    else:
        norm_ti1_use, norm_vws_use = norm_ti1_ms, norm_vws_ms

    # TI₁
    ax1 = axes[i, 0]
    im1 = ti1_freq_por_cat[cat].clip(min=vmin_ti1_ms).plot(
        ax=ax1, transform=ccrs.PlateCarree(),
        cmap=cmap_ti1, norm=norm_ti1_use,
        add_colorbar=False
    )
    ax1.coastlines(); ax1.add_feature(cfeature.BORDERS, linewidth=0.5)
    ax1.set_extent([285.5, 353.5, 36.5, 67.5])
    ax1.set_title(f"(A{i+1}) TI₁ – {cat}", fontsize=8, pad=3)
    ax1.plot(lons, lats, transform=ccrs.Geodetic(), color="black", linewidth=0.6)

    # VWS
    ax2 = axes[i, 1]
    im2 = VWS_freq_por_cat[cat].clip(min=vmin_vws_ms).plot(
        ax=ax2, transform=ccrs.PlateCarree(),
        cmap=cmap_vws, norm=norm_vws_use,
        add_colorbar=False
    )
    ax2.coastlines(); ax2.add_feature(cfeature.BORDERS, linewidth=0.5)
    ax2.set_extent([285.5, 353.5, 36.5, 67.5])
    ax2.set_title(f"(B{i+1}) VWS – {cat}", fontsize=8, pad=3)
    ax2.plot(lons, lats, transform=ccrs.Geodetic(), color="black", linewidth=0.6)

# Sin etiquetas
for ax in axes.flat:
    ax.set_xticks([]); ax.set_yticks([]); ax.set_xlabel(""); ax.set_ylabel("")

# Barras de color (4 total, bien separadas)
# Lineales (Ligera)
cax_lin_ti1 = fig.add_axes([0.09, 0.12, 0.33, 0.018])
cb_lin_ti1 = plt.colorbar(
    plt.cm.ScalarMappable(norm=norm_ti1_light, cmap=cmap_ti1),
    cax=cax_lin_ti1, orientation="horizontal"
)
cb_lin_ti1.set_label("% de días DJF (TI₁ – Ligera)")

cax_lin_vws = fig.add_axes([0.58, 0.12, 0.33, 0.018])
cb_lin_vws = plt.colorbar(
    plt.cm.ScalarMappable(norm=norm_vws_light, cmap=cmap_vws),
    cax=cax_lin_vws, orientation="horizontal"
)
cb_lin_vws.set_label("% de días DJF (VWS – Ligera)")

# Logarítmicas (Moderada/Severa)
cax_log_ti1 = fig.add_axes([0.09, 0.025, 0.33, 0.018])
cb_log_ti1 = plt.colorbar(
    plt.cm.ScalarMappable(norm=norm_ti1_ms, cmap=cmap_ti1),
    cax=cax_log_ti1, orientation="horizontal",
    format=LogFormatterMathtext()
)
cb_log_ti1.set_label("% de días DJF (TI₁ – Moderada/Severa)")
cb_log_ti1.locator = LogLocator(base=10, numticks=4)
cb_log_ti1.update_ticks()

cax_log_vws = fig.add_axes([0.58, 0.025, 0.33, 0.018])
cb_log_vws = plt.colorbar(
    plt.cm.ScalarMappable(norm=norm_vws_ms, cmap=cmap_vws),
    cax=cax_log_vws, orientation="horizontal",
    format=LogFormatterMathtext()
)
cb_log_vws.set_label("% de días DJF (VWS – Moderada/Severa)")
cb_log_vws.locator = LogLocator(base=10, numticks=4)
cb_log_vws.update_ticks()

# Título global
plt.suptitle("Frecuencia multimodelo por tipo de turbulencia – DJF 2030–2100",
             fontsize=12, y=0.995)
plt.tight_layout(rect=[0, 0.16, 1, 0.98])  # más margen inferior

plt.savefig(os.path.join(base_out, f"TI1_VWS_frecuencia_por_categoria_{escenario}.png"),
            dpi=300, bbox_inches="tight")
plt.close(fig)

print("✔ Figura final con leyendas bien separadas guardada correctamente.")

# GUARDAR RESULTADOS MULTIMODELO EN NetCDF (TI₁ y VWS)
# TI₁
# 1. Tendencias multimodelo de TI₁
ti1_mean.to_netcdf(os.path.join(base_out, f"TI1_tendencia_multimodelo_{escenario}.nc"))
print(f"Archivo guardado: TI1_tendencia_multimodelo_{escenario}.nc")

# 2. Climatología media multimodelo de TI₁
ti1_clim_mean.to_netcdf(os.path.join(base_out, f"TI1_media_multimodelo_{escenario}.nc"))
print(f"Archivo guardado: TI1_media_multimodelo_{escenario}.nc")

# VWS
# 3. Tendencia multimodelo de cizalladura vertical
VWS_mean.to_netcdf(os.path.join(base_out, f"VWS_tendencia_multimodelo_{escenario}.nc"))
print(f"Archivo guardado: VWS_tendencia_multimodelo_{escenario}.nc")

# 4. Media multimodelo de VWS
VWS_multimodel_mean.to_netcdf(os.path.join(base_out, f"VWS_media_multimodelo_{escenario}.nc"))
print(f"Archivo guardado: VWS_media_multimodelo_{escenario}.nc")

print("Figura final con leyendas bien separadas guardada correctamente.")

In [ ]:
# CÁLCULOS DE VIENTO ZONAL POR MODELO
import pandas as pd
import matplotlib.ticker as mticker

tendencias_modelos_barras = []
mapas_modelos = []

# Calcular límites globales de ejes y colorbar
limites_barras, limites_mapas = [], []

for modelo, ua in zip(modelos, ua_models):
    print(f"Calculando viento zonal para {modelo}...")

    # agrupar por inviernos
    ua_with_year = asignar_invierno(ua)
    ua_djf_yearly = ua_with_year.groupby("winter_year").mean("time")

    # tendencia (m/s por año)
    ua_trend = ua_djf_yearly.polyfit(dim="winter_year", deg=1)
    ua_slope = ua_trend["polyfit_coefficients"].sel(degree=1)
    ua_slope_decadal = ua_slope * 10  # pasar a m/s por década

    # añadir límites globales (para escalas comunes)
    limites_barras += [
        float(ua_slope_decadal.mean(("lat", "lon")).min()),
        float(ua_slope_decadal.mean(("lat", "lon")).max())
    ]
    limites_mapas += [
        float(ua_slope_decadal.sel(plev=25000, method="nearest").min()),
        float(ua_slope_decadal.sel(plev=25000, method="nearest").max())
    ]

    ua_slope_profile = ua_slope_decadal.mean(dim=("lat", "lon"))
    ua_map_250 = ua_slope_decadal.sel(plev=25000, method="nearest")

    # Añadir la dimensión 'modelo' solo si no existe ya
    if "modelo" not in ua_slope_profile.dims:
        ua_slope_profile = ua_slope_profile.expand_dims(modelo=[modelo])
    else:
        ua_slope_profile = ua_slope_profile.assign_coords(modelo=[modelo])
    
    tendencias_modelos_barras.append(ua_slope_profile)
    
    if "modelo" not in ua_map_250.dims:
        ua_map_250 = ua_map_250.expand_dims(modelo=[modelo])
    else:
        ua_map_250 = ua_map_250.assign_coords(modelo=[modelo])
    
    mapas_modelos.append(ua_map_250)

# calcular límites globales comunes más realistas
# (basado en perfiles promediados verticalmente)
todas_barras = xr.concat(tendencias_modelos_barras, dim="modelo")
max_barras = float(abs(todas_barras).max())

todas_mapas = xr.concat(mapas_modelos, dim="modelo")
max_mapas = float(abs(todas_mapas).max())

lim_global_barras = np.round(max_barras * 1.1, 2)  # 10% margen
lim_global_mapas = np.round(max_mapas * 1.1, 2)

print(f"Escalas globales ajustadas -> Barras: ±{lim_global_barras:.2f}, Mapas: ±{lim_global_mapas:.2f}")

# MEDIA Y DESVIACIÓN ESTÁNDAR MULTIMODELO
tendencias_ds = xr.concat(tendencias_modelos_barras, dim="modelo")
tendencia_media = tendencias_ds.mean("modelo")
tendencia_std   = tendencias_ds.std("modelo")

# perfiles
tend_plevs = tendencia_media["plev"].values / 100  # hPa
tend_vals  = np.squeeze(tendencia_media.values)    # eliminar dimensiones sobrantes
tend_err   = np.squeeze(tendencia_std.values)
print("Dimensiones tendencia_media:", tendencia_media.dims)
print("Shape tendencia_media:", tendencia_media.shape)

# Asegurar arrays 1D sin NaN y sin duplicados reales
tend_plevs = np.array(tend_plevs).flatten()
tend_vals  = np.array(tend_vals).flatten()
tend_err   = np.array(tend_err).flatten()

mask = np.isfinite(tend_vals)
tend_plevs, tend_vals, tend_err = tend_plevs[mask], tend_vals[mask], tend_err[mask]

# Redondear niveles de presión para agrupar (por si hay diferencias menores)
plevs_redondeados = np.round(tend_plevs, 0)
df = pd.DataFrame({"plev": plevs_redondeados, "val": tend_vals, "err": tend_err})
df = df.groupby("plev", as_index=False).mean()  # promedio si hay duplicados

# Orden descendente (de superficie a alta atmósfera)
df = df.sort_values("plev", ascending=False)

tend_plevs = df["plev"].values
tend_vals  = df["val"].values
tend_err   = df["err"].values

# Colores según el signo del valor
colors = np.where(tend_vals > 0, "red", "blue")

# FIGURA MULTIMODELO BARRAS + ERROR
fig, ax = plt.subplots(figsize=(3.5, 5))
height = 40

# Dibujar cada barra de forma individual con color y error unidireccional
for p, val, err in zip(tend_plevs, tend_vals, tend_err):
    color = "red" if val > 0 else "blue"
    
    # Error unidireccional (solo hacia el lado del valor)
    if val >= 0:
        xerr_plot = [[0], [err]]  # solo hacia la derecha
    else:
        xerr_plot = [[err], [0]]  # solo hacia la izquierda

    ax.barh(
        p, val,
        color=color, height=height,
        xerr=xerr_plot, ecolor="k", capsize=2, alpha=0.8
    )

ax.axvline(0, color="k", linewidth=1)
ax.invert_yaxis()
ax.set_yticks(tend_plevs)
ax.set_yticklabels(tend_plevs.astype(int))

ax.set_xlabel("m/s por década", fontsize=9)
ax.set_xlim(-lim_global_barras, lim_global_barras)
ax.set_ylabel("Nivel de presión (hPa)", fontsize=9)
ax.set_title(f"Tendencia de velocidad zonal DJF (2030–2100)\nMedia multimodelo ± 1σ ({escenario})", fontsize=10)

ax.xaxis.set_major_locator(mticker.MultipleLocator(0.2))
ax.tick_params(axis="both", which="major", labelsize=8.5)

os.makedirs(base_out2, exist_ok=True)
out_file = os.path.join(base_out2, "media_multimodelo_barras_ua_tendencia1.png")
fig.savefig(out_file, bbox_inches="tight", dpi=300)
plt.close(fig)
print(f"Figura multimodelo (barras) guardada en {out_file}")

# MAPA MULTIMODELO A 250 hPa
mapas_ds = xr.concat(mapas_modelos, dim="modelo")
mapa_media = mapas_ds.mean("modelo")

fig = plt.figure(figsize=(7, 4))
ax = plt.axes(projection=ccrs.PlateCarree())

norm = mcolors.TwoSlopeNorm(
    vmin=float(mapa_media.min()),
    vcenter=0,
    vmax=float(mapa_media.max())
)

im = mapa_media.plot(
    ax=ax, transform=ccrs.PlateCarree(),
    cmap="RdBu_r", norm=norm,
    cbar_kwargs={
        "label": "Tendencia (m/s por década)",
        "orientation": "horizontal",
        "pad": 0.1,
        "shrink": 0.7
    }
)

ax.coastlines()
ax.add_feature(cfeature.BORDERS, linewidth=0.5)
ax.set_extent([280, 352.5, 35, 69], crs=ccrs.PlateCarree())

# Añadir etiquetas de lat/lon
gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                  linewidth=0.5, color='gray', alpha=0.5, linestyle='--')

# Configuración de etiquetas
gl.top_labels = False     # Quita etiquetas arriba
gl.right_labels = False   # Quita etiquetas a la derecha
gl.xformatter = LONGITUDE_FORMATTER
gl.yformatter = LATITUDE_FORMATTER
gl.xlabel_style = {"size": 8}
gl.ylabel_style = {"size": 8}

# Añadir la ruta DUB–JFK
puntos = geod.npts(lon_DUB, lat_DUB, lon_JFK, lat_JFK, 100)
lons = [lon_DUB] + [p[0] for p in puntos] + [lon_JFK]
lats = [lat_DUB] + [p[1] for p in puntos] + [lat_JFK]
ax.plot(lons, lats, transform=ccrs.Geodetic(),
        color="black", linewidth=0.8, linestyle="-")

plt.title(f"Tendencia multimodelo de velocidad zonal a 250 hPa\nDJF 2030–2100 ({escenario})", fontsize=9)
os.makedirs(base_out2, exist_ok=True)
out_file = os.path.join(base_out2, "media_multimodelo_mapa_ua_tendencia1.png")
plt.savefig(out_file, bbox_inches="tight", dpi=300)
plt.close(fig)
print(f"Figura multimodelo (mapa) guardada en {out_file}")

# FIGURA CON TODOS LOS GRÁFICOS DE BARRAS (MODELOS)
print("Creando figura combinada con todas las barras...")

n_mod = len(tendencias_modelos_barras)
ncols = 2  # puedes ajustar (5 o 4 según tu número de modelos)
nrows = math.ceil(n_mod / ncols)

fig, axes = plt.subplots(nrows, ncols, figsize=(ncols*3.5, nrows*4.5), sharex=True, sharey=True)
axes = axes.flatten()

for i, (modelo, profile) in enumerate(zip(modelos, tendencias_modelos_barras)):
    ax = axes[i]
    plevs = profile["plev"].values / 100
    vals = np.squeeze(profile.values)  # elimina dimensiones sobrantes (como 'modelo')
    colors = np.where(vals > 0, "red", "blue")  # vectorizado, sin bucle
    ax.barh(plevs, vals, color=colors, height=40)
    ax.axvline(0, color="k", linewidth=1)
    ax.invert_yaxis()
    ax.set_xlim(-lim_global_barras, lim_global_barras)
    ax.set_title(modelo, fontsize=8)
    ax.tick_params(axis="both", which="major", labelsize=7)

# Ocultar ejes vacíos si sobran
for j in range(i+1, len(axes)):
    fig.delaxes(axes[j])

fig.suptitle(f"Tendencia viento zonal DJF (2030–2100, {escenario}) – Todos los modelos", fontsize=11)
fig.text(0.5, 0.04, "m/s por década", ha="center")
fig.text(0.04, 0.5, "Nivel de presión (hPa)", va="center", rotation="vertical")

out_file = os.path.join(base_out2, "ua_todos_modelos_barras.png")
plt.savefig(out_file, bbox_inches="tight", dpi=300)
plt.close(fig)
print(f"Figura combinada de barras guardada: {out_file}")

# FIGURA CON TODOS LOS MAPAS (MODELOS)
print("Creando figura combinada con todos los mapas...")

n_mod = len(mapas_modelos)
ncols = 2  # ajusta si quieres 4 o 5 por fila
nrows = math.ceil(n_mod / ncols)

fig, axes = plt.subplots(
    nrows, ncols, figsize=(ncols*3.5, nrows*3.2),
    subplot_kw={"projection": ccrs.PlateCarree()}
)
axes = axes.flatten()

norm = mcolors.TwoSlopeNorm(vmin=-lim_global_mapas, vcenter=0, vmax=lim_global_mapas)

for i, (modelo, mapa) in enumerate(zip(modelos, mapas_modelos)):
    ax = axes[i]
    mapa.plot(
        ax=ax, transform=ccrs.PlateCarree(),
        cmap="RdBu_r", norm=norm,
        add_colorbar=False
    )
    ax.coastlines(); ax.add_feature(cfeature.BORDERS, linewidth=0.5)
    ax.set_extent([280, 352.5, 35, 69], crs=ccrs.PlateCarree())
    ax.set_title(modelo, fontsize=8)
    ax.set_xticks([]); ax.set_yticks([])

# Ejes vacíos
for j in range(i+1, len(axes)):
    fig.delaxes(axes[j])

# Colorbar común
cbar_ax = fig.add_axes([0.25, 0.05, 0.5, 0.03])
sm = plt.cm.ScalarMappable(cmap="RdBu_r", norm=norm)
fig.colorbar(sm, cax=cbar_ax, orientation="horizontal", label="Tendencia (m/s por década)")

fig.suptitle(f"Viento zonal a 250 hPa – Todos los modelos ({escenario})", fontsize=11)
out_file = os.path.join(base_out2, "ua_todos_modelos_mapas.png")
plt.savefig(out_file, bbox_inches="tight", dpi=300)
plt.close(fig)
print(f"Figura combinada de mapas guardada: {out_file}")